# Copy Number Pipeline

In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
from IPython.display import Image,display
import dalmatian as dm
from IPython.core.display import HTML 
from src.CCLE_postp_function import *
import sys
sys.path.insert(0, '../JKBio/')
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()

In [ ]:
cwd = os.getcwd()
print(cwd)

In [ ]:
namespace1="broad-genomics-delivery"
workspace1="Getz_IBM_CellLines_Exomes"
namespace2="broad-firecloud-ccle"
workspace2="CCLE_DepMap_WES"
namespace3="broad-genomics-delivery"
workspace3="CCLE_DepMap_WES"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_WES_CN_hg38"
source1="ibm"
source2="ccle"
source3="ccle"
source4="ccle"
sample_set_id = "19Q4"
release = sample_set_id

In [ ]:
wm1 = dm.WorkspaceManager(namespace1, workspace1)
wm2 = dm.WorkspaceManager(namespace2, workspace2)
wm3 = dm.WorkspaceManager(namespace3, workspace3)
# wm4 = dm.WorkspaceManager(namespace4, workspace4)

refwm = dm.WorkspaceManager(refnamespace, refworkspace)

In [ ]:
## made some changes; now most all TerraFunction functions take a string of "namespace/workspace" 
## instead of the workspace manager object
## addToSampleSet, addToPairSet, updateAllSampleSet, waitForSubmission...
name_work1 = namespace1 + '/' + workspace1
name_work2 = namespace2 + '/' + workspace2
name_work3 = namespace3 + '/' + workspace3
refname_work = refnamespace + '/' + refworkspace

In [ ]:
# just want to update this sample set with a few lines we missed
toKeep = refwm.get_sample_sets()["samples"][sample_set_id] 
len(toKeep)
toKeep

In [ ]:
newsample = createDatasetWithNewCellLines(refwm, sample_set_id, wm3, source3, dry_run = True, forcekeep = toKeep)

In [ ]:
newsample

In [ ]:
# this gives us the list of samples that exist in the sample set
print(len(refwm.get_sample_sets()["samples"][sample_set_id]))
refwm.get_sample_sets()["samples"][sample_set_id] 

# ReaLign in HG38

In [ ]:
othersamples = ['ACH-001172_2',
'ACH-001807_1',
'ACH-001613_1',
'ACH-002508_1']

In [ ]:
terra.addToSampleSet(refname_work,sample_set_id,othersamples)

In [ ]:
# this gives us the list of samples that exist in the sample set
print(len(refwm.get_sample_sets()["samples"][sample_set_id]))
refwm.get_sample_sets()["samples"][sample_set_id] 

In [ ]:
samples_df = refwm.get_samples()
refwm.delete_entity_attributes('sample', samples_df['readgroup_ubams'])

In [ ]:
bamtoubam= "BamToUnmappedRGBams_MC"
ubamtofilelist = "Generate_uBAM_File_List"
realign="Realign_WES_GATK4"

In [ ]:
bamtoubam = refwm.get_config(bamtoubam)

In [ ]:
refwm.update_config(bamtoubam)
subid = refwm.create_submission(bamtoubam['name'],sample_set_id,"sample_set","this.samples", use_callcache=True)

In [ ]:
terra.waitForSubmission(refname_work, subid)

In [ ]:
ubamtofilelist = refwm.get_config(ubamtofilelist)
ubamtofilelist

In [ ]:
# currently, v1 works but v5 of the method fails. I'm not sure what's going on here...
refwm.update_config(ubamtofilelist)
subid = refwm.create_submission(ubamtofilelist['name'],sample_set_id,"sample_set","this.samples")

In [ ]:
terra.waitForSubmission(refname_work, subid)

In [ ]:
realign = refwm.get_config(realign)
realign

In [ ]:
refwm.update_config(realign)
subid = refwm.create_submission(realign['name'],sample_set_id,"sample_set","this.samples")

In [ ]:
terra.waitForSubmission(refname_work, subid)

In [ ]:
refwm.create_submission()

In [ ]:
CNV_woXX = refwm.get_config("CNV_sample_XX")
CNV_woXX

In [ ]:
refwm.update_config(CNV_woXX)
submission_id = refwm.create_submission(CNV_woXX['name'],etype='sample_set',entity=sample_set_id,expression='this.samples')

In [ ]:
terra.waitForSubmission(refname_work,submission_id)

In [ ]:
aggregate = refwm.get_config("Aggregate_CN_seg_files")
aggregate

In [ ]:
submission_id = refwm.create_submission(aggregate['name'],entity=sample_set_id)

In [ ]:
terra.waitForSubmission(refname_work,submission_id)

In [ ]:
aggregated = refwm.get_entities('sample_set').loc[sample_set_id]["combined_seg_file"]
aggregated

In [ ]:
! gsutil cp $aggregated "temp/cnv_ccle.called.seg"

In [ ]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')

In [ ]:
%%R
genome_version <- 'hg38'
release <- '19Q3'
hg38_cyto_band_reference <- 'data/hg38_cytoband.gz'
new_samples_copy_number_broad_wes <- 'temp/cnv_ccle.called.seg'

In [ ]:
%%R
source("CCLE_postp_function.R")

In [ ]:
%%R
# Previous release copy number profiles. This line will need to be updated as well
wes.priority.cn.seg.profiles <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.file='wes.19Q3.segmented') %>%
  dplyr::select(DepMap_ID, Chromosome, Start, End, Num_Probes, Segment_Mean, Source)

## If want to reprocess something

In [ ]:
#%%R
#segments_unfiltered <- readr::read_csv('temp/wes.19Q3.segmented.cn')

In [ ]:
prevversion = '5c76b219-404f-479a-9b7c-7cf5699fc161_aggregate_CN_segments_wrkflw_c0119f63-73d2-4f47-a6ec-88f24f74f3f2_call-aggregate_CN_segments_19Q3.called.seg'

In [ ]:
%%R
segments <- processSegments(new_samples_copy_number_broad_wes)
segments <- filterForCCLE(segments)
segments <- interpolateGapsInSegmented(segments)$segs
segments <- extendEndsOfSegments(segments,'../JKBio/data/hg38_cytoband.gz')
print(segments)
print(head(wes.priority.cn.seg.profiles))
segments_unfiltered <- reprioritizeData(segments, wes.priority.cn.seg.profiles)
# Fill in the gaps on the entire dataset
# Extend start sites to 1, end sites to the end of the chromosome?

In [ ]:
%%R
embargoed <- "WES_embargoed.txt"
blacklisted <- "blacklist.txt"

In [ ]:
%%R
# TODO: IF seqnames (CHR) are 1-9 values, append "chr" in front of each! (bulk)
# ifelse(grepl("[0-9]+", new_copy_number$ChrChromosome), "chr"+, "no")
segments_blacklisted <- filterBlackListedLine(filepath=blacklisted,segments_unfiltered)
segments_embargoed <- filterBlackListedLine(filepath=embargoed,segments_unfiltered)

In [ ]:
%%R
write.table(segments_unfiltered, file = paste0("temp/wes.",release,".segmented.cn1"), sep = ',', quote = F, row.names = F) 

In [ ]:
%%R
entrezgenes <- generateEntrezGenes()

In [ ]:
%%R
head(corner(entrezgenes))

In [ ]:
%%R
res <- generateGeneLevelMatrixFromSegments(entrezgenes, segments_unfiltered)

In [ ]:
%%R
genematrix_unfiltered <- res$gene_level_data_hg38
corner(genematrix_unfiltered)

In [ ]:
%%R
write.table(genematrix_unfiltered, file = paste0('temp/wes.',release,'.gene.cn'), 
sep = ',', quote = F, row.names = T)

## Validation step

In [ ]:
genecn = pd.read_csv('temp/wes.'+release+'.gene.cn', sep = ',')

In [ ]:
prevsegmentcn = pd.read_csv('temp/cnv_ccle.called.seg', sep = '\t')
segmentcn = pd.read_csv('temp/wes.'+release+'.segmented.cn', sep = ',')

In [ ]:
genecn.index = [i[-1] for i in genecn.index.str.split('_').tolist()]
segmentcn["DepMap_ID"] = [i[-1] for i in segmentcn["DepMap_ID"].str.split('_').tolist()]

In [ ]:
embargoed = "WES_embargoed.txt"
embargoed = [line.rstrip('\n') for line in open(embargoed)]
blacklisted = "blacklist.txt"
blacklisted = [line.rstrip('\n') for line in open(blacklisted)]

In [ ]:
prev = set(tc.get(name='segmented-cn-wes-prioritzed-7fe1', version=27, file='wes.19Q3interim.gene').index.values.tolist())
new1 = set(genecn.index.values.tolist())
new2 = set(segmentcn['Sample'].values.tolist())
print(len(prev), len(prev & new1), len(new1), len(new1 & new2))

In [ ]:
checkAmountOfSegments(segmentcn,thresh = 750)

In [ ]:
checkGeneChangeAccrossAll(genecn, thresh=1.5)

In [ ]:
newsamples =  list(set(segmentcn["Sample"].tolist()))

In [ ]:
segmentcn[segmentcn["DepMap_ID"]=="ACH-000658"]

In [ ]:
newsamples= refwm.get_sample_sets().loc[sample_set_id].samples

In [ ]:
samples = refwm.get_samples()
plots = samples.loc[samples.index.isin(newsamples)]["modeled_segments_plot_tumor"].values
for plot in plots:
    ! gsutil cp $plot temp/

In [ ]:
for plot in plots:
    display(Image('temp/'+plot.split('/')[-1]))

In [ ]:
genecn.to_csv('temp/wes.'+release+'.gene.cn')
segmentcn.to_csv('temp/wes.'+release+'.segmented.cn')

In [ ]:
## for genecn removing first blacklisted, then embargoed, to create two datasets
genecn = genecn.apply(lambda x: np.log2(1+x))
for i in genecn.index:
    if i in blacklisted:
        genecn = genecn.drop(i)
genecn.to_csv('temp/wes.'+release+'.blacklisted.gene.cn')
for i in genecn.index:
    if i in embargoed:
        genecn = genecn.drop(i)
genecn.to_csv('temp/wes.'+release+'.embargoed.gene.cn')

In [ ]:
## for segment removing first blacklisted, then embargoed, to create two datasets
segmentcn['Segment_Mean'] = segmentcn['Segment_Mean'].apply(lambda x: np.log2(1+x))
for i, val in segmentcn.iterrows():
    if val['DepMap_ID'] in blacklisted:
        segmentcn = segmentcn.drop(i)
segmentcn.to_csv('temp/wes.'+release+'.blacklisted.segmented.cn')
for i, val in segmentcn.iterrows():
    if val['DepMap_ID'] in embargoed:
        segmentcn = segmentcn.drop(i)
segmentcn.to_csv('temp/wes.'+release+'.embargoed.segmented.cn')

In [ ]:
segmentcn

# Upload to taiga

In [ ]:
tc.update_dataset(dataset_permaname="segmented-cn-wes-prioritzed-7fe1", 
                  upload_file_path_dict={'temp/wes.'+release+'.gene.cn': 'NumericMatrixCSV',
                                        'temp/wes.'+release+'.segmented.cn': 'TableCSV',
                                        'temp/wes.'+release+'.embargoed.gene.cn': 'NumericMatrixCSV','temp/wes.'+release+'.embargoed.segmented.cn': 'TableCSV',
                                         'temp/wes.'+release+'.blacklisted.gene.cn': 'NumericMatrixCSV',
                                         'temp/wes.'+release+'.blacklisted.segmented.cn': 'TableCSV'},
                dataset_description="updating to "+release+". Some cells lines have been flagged as: \n \
                - having bad looking copy ration plots = [] \n \
                 - having too many segments = [] \n \
                  - Genes having a similar CN value accross all []",
                 )